In [1]:
import numpy as np
import cv2
import timm
import tqdm

import torch
from torchvision import datasets, transforms
from torch.utils import data

from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, confusion_matrix
from PIL import Image

from matplotlib import pyplot as plt


np.random.seed(42)
NUM_CLASSES = 9
BATCH_SIZE = 32
device = torch.device("cuda")
PATH_TO_TEST_DATASET = "data/CRC-VAL-HE-7K/"


def cv2_loader(path: str):
    return Image.fromarray(cv2.imread(path, -1)[:, :, ::-1])


def cv2_jpeg80_damage_loader(path: str):
    img = cv2.imread(path, -1)
    quality = 80
    _, img = cv2.imencode('test.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, quality))
    img = cv2.imdecode(img, cv2.IMREAD_UNCHANGED)
    return Image.fromarray(img[:, :, ::-1])

def cv2_jpeg60_damage_loader(path: str):
    img = cv2.imread(path, -1)
    quality = 60
    _, img = cv2.imencode('test.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, quality))
    img = cv2.imdecode(img, cv2.IMREAD_UNCHANGED)
    return Image.fromarray(img[:, :, ::-1])


def cv2_jpeg40_damage_loader(path: str):
    img = cv2.imread(path, -1)
    quality = 40
    _, img = cv2.imencode('test.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, quality))
    img = cv2.imdecode(img, cv2.IMREAD_UNCHANGED)
    return Image.fromarray(img[:, :, ::-1])


def cv2_jpeg20_damage_loader(path: str):
    img = cv2.imread(path, -1)
    quality = 20
    _, img = cv2.imencode('test.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, quality))
    img = cv2.imdecode(img, cv2.IMREAD_UNCHANGED)
    return Image.fromarray(img[:, :, ::-1])


/home/user/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/user/anaconda3/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-07-19 19:33:10.532597: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2024-07-19 19:33:10.532612: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model = timm.create_model('efficientnet_b0.ra_in1k', pretrained=True, num_classes=NUM_CLASSES, in_chans=3, global_pool='avg')
model.load_state_dict(torch.load('./0.9774_0.9688.pt'))
model.to(device)
model.eval()

model2 = timm.create_model('efficientnet_b0.ra_in1k', pretrained=True, num_classes=NUM_CLASSES, in_chans=3, global_pool='max')
model2.load_state_dict(torch.load('./97_39_96_20.pt'))
model2.to(device)
model2.eval()

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
      

In [3]:
def evaluate(model, test_dataloader):
    print("Running Evaluation...")

    targets_array = []
    predictions_array = []

    with torch.no_grad():

        test_iter = iter(test_dataloader)
        for j in tqdm.tqdm(range(len(test_dataloader))):

            image, labels = next(test_iter)
            image = image.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            predictions = model(image)
            predictions2 = model(image.flip(2))
            predictions3 = model(image.flip(3))
            predictions23 = model(image.flip(2).flip(3))
            
            
            predictions_2 = model2(image)
            predictions2_2 = model2(image.flip(2))
            predictions3_2 = model2(image.flip(3))
            predictions23_2 = model2(image.flip(2).flip(3))
            
            _, predictions = torch.max(
                predictions.data + predictions2.data + predictions3.data  + predictions23.data +
                predictions_2.data + predictions2_2.data + predictions3_2.data  + predictions23_2.data
                , 1)

            predictions = predictions.detach().cpu().numpy()
            targets = labels.detach().cpu().numpy()

            for k in range(targets.shape[0]):

                target = targets[k]
                predicted = predictions[k]

                targets_array.append(target)
                predictions_array.append(predicted)

        print("Accuracy: " + str(accuracy_score(targets_array, predictions_array)))
        print("Balanced Accuracy: " + str(balanced_accuracy_score(targets_array, predictions_array)))
        

        print(classification_report(targets_array, predictions_array))
        print(confusion_matrix(targets_array, predictions_array))
        
        
        return predictions_array

In [4]:
test_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_dataset = datasets.ImageFolder(PATH_TO_TEST_DATASET, transform=test_transforms, loader=cv2_loader)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False, drop_last=False)

predictions_no_jpeg_aug = evaluate(model, test_dataloader)

Running Evaluation...


100%|████████████████████████████████████████████████████████████████████████████████████████| 225/225 [01:00<00:00,  3.71it/s]

Accuracy: 0.9832869080779945
Balanced Accuracy: 0.9744136831688599
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1338
           1       1.00      1.00      1.00       847
           2       0.99      1.00      0.99       339
           3       0.99      1.00      0.99       634
           4       0.99      1.00      0.99      1035
           5       0.91      0.98      0.94       592
           6       0.99      0.98      0.99       741
           7       0.97      0.83      0.89       421
           8       0.99      0.99      0.99      1233

    accuracy                           0.98      7180
   macro avg       0.98      0.97      0.98      7180
weighted avg       0.98      0.98      0.98      7180

[[1333    1    0    0    3    1    0    0    0]
 [   0  847    0    0    0    0    0    0    0]
 [   0    0  338    0    0    1    0    0    0]
 [   0    0    0  634    0    0    0    0    0]
 [   3    0    0    0 1031    0    0

In [5]:
test_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_dataset = datasets.ImageFolder(PATH_TO_TEST_DATASET, transform=test_transforms, loader=cv2_jpeg20_damage_loader)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False, drop_last=False)

predictions_jpeg20_aug = evaluate(model, test_dataloader)

Running Evaluation...


100%|████████████████████████████████████████████████████████████████████████████████████████| 225/225 [01:01<00:00,  3.67it/s]

Accuracy: 0.9759052924791086
Balanced Accuracy: 0.9646702776412445
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1338
           1       1.00      1.00      1.00       847
           2       0.99      0.99      0.99       339
           3       0.99      1.00      0.99       634
           4       0.96      1.00      0.98      1035
           5       0.91      0.94      0.93       592
           6       0.99      0.99      0.99       741
           7       0.88      0.81      0.84       421
           8       0.99      0.98      0.98      1233

    accuracy                           0.98      7180
   macro avg       0.97      0.96      0.97      7180
weighted avg       0.98      0.98      0.98      7180

[[1329    4    0    0    5    0    0    0    0]
 [   0  847    0    0    0    0    0    0    0]
 [   0    0  334    0    1    1    0    3    0]
 [   0    0    0  631    0    0    0    3    0]
 [   1    0    0    0 1033    0    0

In [6]:
test_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_dataset = datasets.ImageFolder(PATH_TO_TEST_DATASET, transform=test_transforms, loader=cv2_jpeg40_damage_loader)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False, drop_last=False)

predictions_jpeg40_aug = evaluate(model, test_dataloader)

Running Evaluation...


100%|████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:59<00:00,  3.77it/s]

Accuracy: 0.9778551532033426
Balanced Accuracy: 0.9663962920070883
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1338
           1       1.00      1.00      1.00       847
           2       0.99      0.99      0.99       339
           3       0.99      1.00      0.99       634
           4       0.98      1.00      0.99      1035
           5       0.88      0.97      0.92       592
           6       0.98      0.99      0.99       741
           7       0.93      0.78      0.85       421
           8       1.00      0.98      0.99      1233

    accuracy                           0.98      7180
   macro avg       0.97      0.97      0.97      7180
weighted avg       0.98      0.98      0.98      7180

[[1331    1    0    0    6    0    0    0    0]
 [   0  847    0    0    0    0    0    0    0]
 [   0    0  335    0    0    3    0    1    0]
 [   0    0    0  633    0    0    0    1    0]
 [   3    0    0    0 1031    0    0

In [7]:
test_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_dataset = datasets.ImageFolder(PATH_TO_TEST_DATASET, transform=test_transforms, loader=cv2_jpeg60_damage_loader)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False, drop_last=False)

predictions_jpeg60_aug = evaluate(model, test_dataloader)

Running Evaluation...


100%|████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:59<00:00,  3.75it/s]

Accuracy: 0.9793871866295265
Balanced Accuracy: 0.9695152622558061
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1338
           1       1.00      1.00      1.00       847
           2       0.99      1.00      0.99       339
           3       0.98      1.00      0.99       634
           4       0.98      1.00      0.99      1035
           5       0.89      0.97      0.93       592
           6       0.98      0.99      0.99       741
           7       0.94      0.80      0.86       421
           8       1.00      0.98      0.99      1233

    accuracy                           0.98      7180
   macro avg       0.97      0.97      0.97      7180
weighted avg       0.98      0.98      0.98      7180

[[1330    1    0    0    7    0    0    0    0]
 [   0  847    0    0    0    0    0    0    0]
 [   0    0  338    0    0    1    0    0    0]
 [   0    0    0  633    0    0    0    1    0]
 [   2    0    0    0 1032    0    0

In [8]:
test_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_dataset = datasets.ImageFolder(PATH_TO_TEST_DATASET, transform=test_transforms, loader=cv2_jpeg80_damage_loader)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False, drop_last=False)

predictions_jpeg80_aug = evaluate(model, test_dataloader)

Running Evaluation...


100%|████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:59<00:00,  3.77it/s]

Accuracy: 0.9806406685236769
Balanced Accuracy: 0.9714747522536293
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1338
           1       1.00      1.00      1.00       847
           2       0.99      1.00      0.99       339
           3       0.98      1.00      0.99       634
           4       0.98      1.00      0.99      1035
           5       0.89      0.98      0.94       592
           6       0.99      0.99      0.99       741
           7       0.96      0.80      0.87       421
           8       1.00      0.98      0.99      1233

    accuracy                           0.98      7180
   macro avg       0.98      0.97      0.97      7180
weighted avg       0.98      0.98      0.98      7180

[[1329    1    0    0    7    1    0    0    0]
 [   0  847    0    0    0    0    0    0    0]
 [   0    0  338    0    0    1    0    0    0]
 [   0    0    1  632    0    0    0    1    0]
 [   2    0    0    0 1032    0    0